# CS4042 Data Engineering Group Project

In [1]:
import pandas as pd
import numpy as np
import os

### Preprecessing

In [46]:
sources = ['Player-Advanced-Stats', 'Player-Per-Game-Stats', 'Player-Shooting-Stats', 'Player-Totals-Stats']

# Dictionary for each folder
advanced = {}
per_game = {}
shooting = {}
totals = {}

for src in sources:

    src_path = os.path.join('Datasets', src)

    for file in os.listdir(src_path):
        
        file_path = os.path.join(src_path, file)
        name = file[4:-4] # Name of file

        # Assign DataFrame to correct dict
        if name[-1] == 'd':
            advanced[name] = pd.read_csv(file_path)
        elif name[-1] == 'G':
            per_game[name] = pd.read_csv(file_path)
        elif name[-1] == 'g':
            shooting[name] = pd.read_csv(file_path)
        else:
            totals[name] = pd.read_csv(file_path)


Remove any player whos played less than 5 games.

In [ ]:
for df in advanced.values():
    print(df.shape)
    df.drop(df[df['G'] < 5].index, inplace=True)
    print(df.shape)

(680, 30)
(639, 30)
(736, 30)
(677, 30)
(736, 30)
(689, 30)
(467, 30)
(401, 30)
